# Retrieve all Images and output their Data in an Array

In [1]:
from PIL import Image
import Constants
import os

#Switch to the location where the image files are store
os.chdir(Constants.IMAGE_FILE_LOCATION)

#Stores a list of all of the image files
image_files = None
#Iteratively moves through all of the image files in the folder and appends them to the image_files object
for root, dirs, files in os.walk('.'):
    image_files = files


In [53]:
#Open the first image for testing
image = Image.open(image_files[0])
#Load the image
image_pix = image_file.load()
#The height of the image
image_width = image.size[0]
#The width of the image
image_height = image.size[1]

from PIL import ImageEnhance
#Create method to grey scale the image and enhance the contrast so the white colors are brighter and blacks are darker
def enhance_image(image):
    #Grey scale the image
    grey_scale_image = image.convert('L')
    #Enhance the contrast of the image
    contrast_image = ImageEnhance.Contrast(grey_scale_image).enhance(200)
    #Return the modified image
    return contrast_image

In [54]:
contrast_image = enhance_image(image)
contrast_image.show()

In [55]:
#Image only has 3 values now, Super easy to do selective search >:3
print(contrast_image.getcolors())

[(2687839, 0), (3036, 78), (808874, 255)]


In [ ]:
#Create a boolean mask where all the values are set to 0=False first
mask = np.zeros((image_width, image_height))
#Iterate through every pixel in the image and get the RGB sum of the pixel
#If the RGB sum of the pixel is within a certain threshold, then set it to 1, otherwise, leave it
#This is to set all of the ocean water images to 1 and everything else to 0
for i in range(image_width*image_height):
    x_cor = int(i // x.shape[0])
    y_cor = int(i % x.shape[0])
    